In [4]:
import os
import sys


def setup_project_root(start_path='.'):
    """Find the project root, set it as the current working directory, and add it to sys.path."""
    current_path = os.path.abspath(start_path)
    while True:
        if '.git' in os.listdir(current_path):
            project_root = current_path
            break
        parent_path = os.path.dirname(current_path)
        if parent_path == current_path:  # We've reached the root directory
            raise Exception("Could not find project root (.git directory not found)")
        current_path = parent_path
    
    # Change the current working directory to the project root
    os.chdir(project_root)
    print(f"Current working directory set to: {os.getcwd()}")

    # Add project root to sys.path if it's not already there
    if project_root not in sys.path:
        sys.path.insert(0, project_root)
        print(f"Added {project_root} to sys.path")

# sets the current working directory to the project root
setup_project_root()

# Don't cache imports
%load_ext autoreload
%autoreload 2


Current working directory set to: /vol/bitbucket/dm2223/info-theory-experiments


In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from info_theory_experiments.custom_datasets import ResidualStreamDatasetAlongResidStremDirection_1_to_2
from info_theory_experiments.models import GeneralSmileMIEstimator

class MLPEncoder(nn.Module):
    def __init__(self, input_dim=128, latent_dim=1):
        super(MLPEncoder, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim)
        )

    def forward(self, x):
        return self.encoder(x)

class MLPDecoder(nn.Module):
    def __init__(self, latent_dim=1, output_dim=128):
        super(MLPDecoder, self).__init__()
        self.latent_dim = latent_dim
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim)
        )

    def forward(self, x):
        return self.decoder(x)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = ResidualStreamDatasetAlongResidStremDirection_1_to_2()

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

# Initialize the models, loss function, and optimizers
encoder = MLPEncoder().to(device)
decoder = MLPDecoder().to(device)
mse_loss = nn.MSELoss()
optimizer_encoder = optim.Adam(encoder.parameters(), lr=1e-4)
optimizer_decoder = optim.Adam(decoder.parameters(), lr=1e-3)

decoupled_estimator = GeneralSmileMIEstimator(
    x_dim=1,
    y_dim=1,
    critic_output_size=32,
    x_critics_hidden_sizes=[256, 256],
    y_critics_hidden_sizes=[256, 256],
    clip=5,
    include_bias=True,
).to(device)

decoupled_optimizer = optim.Adam(decoupled_estimator.parameters(), lr=1e-3)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    total_decoder_loss = 0
    total_encoder_loss = 0
    for batch_idx, data in enumerate(train_loader):
        data0 = data[:, 0].to(device)
        data1 = data[:, 1].to(device)

        # Forward pass
        data1_pred = decoder(encoder(data0))

        # Decoder loss (minimize MSE)
        decoder_loss = mse_loss(data1_pred, data1)

        # Update decoder
        if batch_idx % 1 == 0:
            optimizer_decoder.zero_grad()
            decoder_loss.backward()
            optimizer_decoder.step()

        # update decoupled estimator
        decoupled_optimizer.zero_grad()
        v0 = encoder(data0)
        v1 = encoder(data1)
        mi = decoupled_estimator(v0, v1)
        decoupled_loss = -mi # maximize MI
        decoupled_loss.backward()
        decoupled_optimizer.step()

        # Recompute the forward pass for encoder loss
        if batch_idx % 1 == 0:
            optimizer_encoder.zero_grad()
            data1_pred = decoder(encoder(data0))
            v0 = encoder(data0)
            v1 = encoder(data1)
            mi = decoupled_estimator(v0, v1)
            encoder_loss = - 0.1 * mi - mse_loss(data1_pred, data1)
            # Update encoder
            optimizer_encoder.zero_grad()
            encoder_loss.backward()
            optimizer_encoder.step()
        
        total_decoder_loss += decoder_loss.item()
        total_encoder_loss += encoder_loss.item()
    
    # Print epoch statistics
    avg_decoder_loss = total_decoder_loss / len(train_loader)
    avg_encoder_loss = total_encoder_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Decoder Loss: {avg_decoder_loss:.4f}, Encoder Loss: {avg_encoder_loss:.4f}, MI: {mi.item():.4f}')

/vol/bitbucket/dm2223/info-theory-experiments/info_theory_experiments/custom_datasets.py:190: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(f"activations2/

Epoch [1/50], Decoder Loss: 0.9999, Encoder Loss: -0.9997, MI: 0.0000
Epoch [2/50], Decoder Loss: 1.0000, Encoder Loss: -0.9999, MI: -0.0000
Epoch [3/50], Decoder Loss: 1.0004, Encoder Loss: -1.0022, MI: 0.1083
Epoch [4/50], Decoder Loss: 0.9834, Encoder Loss: -1.0404, MI: 0.8565
Epoch [5/50], Decoder Loss: 0.9872, Encoder Loss: -1.0832, MI: 0.0790
Epoch [6/50], Decoder Loss: 0.9855, Encoder Loss: -1.1210, MI: 1.8183
Epoch [7/50], Decoder Loss: 0.9840, Encoder Loss: -1.1792, MI: 1.9174
Epoch [8/50], Decoder Loss: 0.9801, Encoder Loss: -1.1968, MI: 1.2285
Epoch [9/50], Decoder Loss: 0.9834, Encoder Loss: -1.1715, MI: 2.1319
Epoch [10/50], Decoder Loss: 0.9860, Encoder Loss: -1.1917, MI: 2.0588
Epoch [11/50], Decoder Loss: 0.9843, Encoder Loss: -1.2232, MI: 2.4428
Epoch [12/50], Decoder Loss: 0.9853, Encoder Loss: -1.2322, MI: 2.8083
Epoch [13/50], Decoder Loss: 0.9870, Encoder Loss: -1.2295, MI: 2.7699
Epoch [14/50], Decoder Loss: 0.9848, Encoder Loss: -1.2491, MI: 4.8115
Epoch [15/50],

In [16]:
# Train decoder with MSE loss using the trained encoder

# Freeze encoder parameters
for param in encoder.parameters():
    param.requires_grad = False

# Initialize decoder (using the MLPDecoder class from the notebook)
decoder = MLPDecoder(latent_dim=encoder.latent_dim, output_dim=128).to(device)

# Define optimizer for decoder
optimizer_decoder = optim.Adam(decoder.parameters(), lr=1e-3)

# Define loss function
mse_loss = nn.MSELoss()

num_epochs = 10  # Adjust as needed
for epoch in range(num_epochs):
    total_loss = 0
    for batch_idx, data in enumerate(train_loader):
        data0 = data[:, 0].to(device)
        data1 = data[:, 1].to(device)

        # Forward pass
        with torch.no_grad():
            encoded = encoder(data0)
        decoded = decoder(encoded)

        # Compute loss
        loss = mse_loss(decoded, data1)

        # Backward pass and optimize
        optimizer_decoder.zero_grad()
        loss.backward()
        optimizer_decoder.step()

        total_loss += loss.item()

    # Print epoch statistics
    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')

# Unfreeze encoder parameters (if needed for future use)
for param in encoder.parameters():
    param.requires_grad = True

print("Decoder training completed.")

# Optionally, you can evaluate the model here
# For example:
encoder.eval()
decoder.eval()
with torch.no_grad():
    test_data = next(iter(train_loader))
    test_input = test_data[:, 0].to(device)
    test_target = test_data[:, 1].to(device)
    encoded = encoder(test_input)
    reconstructed = decoder(encoded)
    test_loss = mse_loss(reconstructed, test_target)
    print(f"Test Loss: {test_loss.item():.4f}")



Epoch [1/10], Average Loss: 0.9968
Epoch [2/10], Average Loss: 0.9937
Epoch [3/10], Average Loss: 0.9926
Epoch [4/10], Average Loss: 0.9901
Epoch [5/10], Average Loss: 0.9882
Epoch [6/10], Average Loss: 0.9854
Epoch [7/10], Average Loss: 0.9834
Epoch [8/10], Average Loss: 0.9820
Epoch [9/10], Average Loss: 0.9824
Epoch [10/10], Average Loss: 0.9806
Decoder training completed.
Test Loss: 0.9836


In [20]:
from info_theory_experiments.trainers import train_feature_network
seed = 3
torch.manual_seed(seed)
config = {
    "torch_seed": seed,
    "dataset_type": "resid_resid_dir",
    "num_atoms": 128,
    "batch_size": 1000,
    "train_mode": False,
    "train_model_B": False,
    "adjust_Psi": True,
    "clip": 5,
    "feature_size": 1,
    "epochs": 2,
    "downward_critics_config": {
        "hidden_sizes_v_critic": [512, 1024, 1024, 512],
        "hidden_sizes_xi_critic": [512, 512, 512],
        "critic_output_size": 32,
        "lr": 1e-3,
        "bias": True,
        "weight_decay": 0,
    },
    
    "decoupled_critic_config": {
        "hidden_sizes_encoder_1": [512, 512, 512],
        "hidden_sizes_encoder_2": [512, 512, 512],
        "critic_output_size": 32,
        "lr": 1e-3,
        "bias": True,
        "weight_decay": 0,
    },
}

project_name = "resid_directoin_1_2-min-mse-max-mi-verification"

_, _ = train_feature_network(
    config=config,
    trainloader=train_loader,
    feature_network_training=encoder,
    project_name=project_name,
)

Training: 100%|██████████| 2/2 [01:04<00:00, 32.08s/it]


Psi,█▇▇▇▆▆▆▆▇▆▇▇▇▇▇▇▇▅▇▆▆▆▄▇▄▅▆▆█▅▆▇▇▆▅▅▁▃▅▄
decoupled_MI,▁▂▂▃▃▃▂▃▄▄▄▅▅▆▅▅▅▅▆▆▇▆▃▅▄▄▆▆▆▆▆▇▇▇█▆▃▅▅▆
downward_MI_0,▆▅▅▅▇▇▇▄▆▇▆▆▁▅▅▅▅▅▅▆▆▆▆▅█▅▆▇▅▆▆█▆▆▇▇▇▆▆▆
downward_MI_1,▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▅▄▄▄▄▅▃▃▁▄▅█▃▅▆▃▇▆▆▆▅▅▅▄
downward_MI_10,▄▄▅▅▄▄▄▄▄▄▄▄▅▄▅▇▅▅▄▅▄▅▅▆▅▅▆▄▄▇▄▄▆▁▆▆▆▆▅█
downward_MI_100,▅▄▄▄▅▅▄▃▄▄▅▅▄▄▄▄▄▅▅▃▅▅▆▅█▆▅▆▃▇▅▄█▆▆█▄▅▆▁
downward_MI_101,▃▃▃▃▅▂▅▆▅▄▄▃▃▄▂▅▄▅▄▆▂▃▇▃▄▄▁▃▄▃▇▃▄▂▆▄█▆▂▄
downward_MI_102,▄▄▄▄▄▇▆▄▄▅▂▅▄▅▃▄▄▄▆▃▅▅▅▂▄▄▆▅▄▆▅▇▁█▆▅▆▆▅▆
downward_MI_103,▃▃▄▆▃▄▄▅█▄▄▃▃▂▄▄▅▄▅▃▄▄▄▇▄▆▄▄▆▃▄▂▃▄▅█▅▅▅▁
downward_MI_104,▃▃▄▃▅▅▃▃▄▄▅▅▃▅▄▅▅▁▂▅▆▃▄▂▄▄█▅▃▅▄▄▃▃▇▅▅▅▆▆
downward_MI_105,▄▄▅▁▄▄▄▄▅▄▄▄▄▄▄▄▄█▅▅▆▃▅▄▄▅▄▄▄▅▅▄▃▆▃▄▆▃▅█
